In [ ]:
import math
import time

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor

from sklearn.model_selection import KFold

import load_data

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def addouter(C, b, factor=1):
    for i, row in enumerate(C):
        for j in range(len(row)):
            row[j] += factor * b[i] * b[j]
    return C

def assert_symmetric(C):
    for i in range(len(C)):
        for j in range(i):
            C[i][j] = C[j][i] = (C[i][j] + C[j][i])/2
    return C

In [ ]:
def get_args(params, scale, classifier):    
    
    args = {arg: params[c]*10**scale[arg] for c, arg in enumerate(scale.keys())}
    
    # Cheating a bit to deal with discrete arguments and integer
    if classifier == LogisticRegression or classifier == LinearRegression:
        args['fit_intercept'] = args['fit_intercept'] > 0.5
    if classifier == MLPClassifier or classifier == MLPRegressor:
        args['max_iter'] = math.ceil(args['max_iter'])
        if args['activation'] < 0.3333:
            args['activation'] = 'logistic'
        elif args['activation'] > 0.6667:
            args['activation'] = 'relu'
        else:
            args['activation'] = 'tanh'
    if classifier == KNeighborsClassifier or classifier == KNeighborsRegressor:
        args['n_neighbors'] = max(round(args['n_neighbors']), 1)
    if classifier == DecisionTreeClassifier or classifier == DecisionTreeRegressor:
        args['min_samples_split'] = float(args['min_samples_split'])
        if args['min_samples_split'] > 1:
            args['min_samples_split'] = math.ceil(args['min_samples_split'])
    if classifier == AdaBoostClassifier or classifier == AdaBoostRegressor:
        args['n_estimators'] = max(round(args['n_estimators']), 1)
        
    return args

In [ ]:
def fitness(X, y, candidate, scale, classifier):
    
    args = get_args(candidate, scale, classifier)
    
    scores = []
    kf = KFold(n_splits=3)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
        clf = classifier(**args)
        clf.fit(X_train, y_train)
        scores.append(clf.score(X_test, y_test))
    
    return -np.mean(scores)   

In [ ]:
class CMA_ES:
    def __init__(self, xstart, scale, clf, max_iter=10, lamb=10, sigma=1):
        self.xstart = xstart
        self.scale = scale
        self.clf = clf
        self.lamb = lamb
        self.sigma = sigma
        
        self.dim = len(xstart)
        self.maxeval = max_iter*self.lamb
        
        self.initialize_parameters()
        
    def initialize_parameters(self):
        N = self.dim
        self.chiN = N**0.5 * (1 - 1./(4*N) + 1. / (21 * N**2))
        self.mu = int(self.lamb/2)
        
        weights = [math.log((self.lamb+1)/2) - math.log(i) for i in range(1, self.lamb+1)]
        self.weights = np.array(weights)/sum(weights[:self.mu])
        self.mueff = sum(self.weights)**2 / sum(self.weights**2)
        
        # Set Adaptation parameters
        self.cc = (4 + self.mueff/N) / (N+4 + 2 * self.mueff/N)
        self.cs = (self.mueff + 2) / (N + self.mueff + 5)
        self.c1 = 2 / ((N + 1.3)**2 + self.mueff)
        self.cmu = min([1-self.c1, 2 * (self.mueff - 2 + 1/self.mueff) / ((N+2)**2 + self.mueff)])
        self.damps = 2 * self.mu/self.lamb + 0.3 + self.cs
        
        self.lazy_gap_evals = 0.5 * N * self.lamb * (self.c1+self.cmu)**-1 / N**2
        self.update_eval = 0
        
    def initialize_variables(self, X, y):
        self.X = X.to_numpy()
        if isinstance(y, pd.Series):
            self.y = y.to_numpy()
        else:
            self.y = y
        self.xmean = self.xstart
        self.pc = self.dim * [0]
        self.ps = self.dim * [0]
        
        self.B = np.eye(self.dim)
        self.D = self.dim * [1]
        self.C = np.eye(self.dim)
        self.invsqrtC = np.eye(self.dim)
        
        self.counteval = 0
        self.best = [self.xmean]
        self.best_fitness = [-fitness(self.X, self.y, self.best[0], self.scale, self.clf)]
        
    def stop(self):
        if self.counteval >= self.maxeval:
            return True
        if len(self.best_fitness) >= 2:
            improvement = abs(self.best_fitness[-1] - self.best_fitness[-2])
            if improvement < 10e-9:
                print(f'Stopping early after {int(self.counteval/self.lamb)} iteration(s)')
                return True
        return False
    
    def propose_candidates(self):
        candidates = []
        for i in range(self.lamb):
            self.counteval += 1
            x = np.random.multivariate_normal(self.xmean, self.sigma*self.C)
            
            # None of the possible values can be negative, so just take the absolute value
            x = [abs(arg) for arg in x]
            
            candidates.append(x)
        return candidates
    
    def sort_candidates(self, X):
        candidate_fitness = []
        for candidate in X:
            candidate_fitness.append(fitness(self.X, self.y, candidate, self.scale, self.clf))
            
        order = np.array(candidate_fitness).argsort()
        X = np.array(X)[order]
        
        self.best.append(X[0])
        self.best_fitness.append(-min(candidate_fitness))
        return X
    
    def update_variables(self, X):
        
        xold = self.xmean
        self.xmean = np.dot(np.transpose(X[:self.mu]), self.weights[:self.mu])
        
        y = np.subtract(self.xmean, xold)
        z = np.dot(self.invsqrtC, y)
        csn = (self.cs * (2 - self.cs) * self.mueff)**0.5 / self.sigma
        for i in range(self.dim):
            self.ps[i] - (1-self.cs) * self.ps[i] + csn * z[i]
        ccn = (self.cc * (2-self.cc) * self.mueff)**0.5 / self.sigma
        
        hsig = (sum(x**2 for x in self.ps) / self.dim / (1-(1-self.cs)**(2*self.counteval/self.lamb)) < 2+4./(self.dim+1))
        for i in range(self.dim):
            self.pc[i] = (1 - self.cc) * self.pc[i] + ccn * hsig * y[i]
        
        c1a = self.c1 * (1 - (1-hsig**2) * self.cc * (2-self.cc))
        self.C = (1 - c1a - self.cmu * sum(self.weights))*self.C
        self.C = addouter(self.C, self.pc, self.c1)
        for k, wk in enumerate(self.weights):
            if wk < 0:
                dx = np.subtract(X[k], xold)
                wk *= self.dim * (self.sigma / sum(xi**2 for xi in np.dot(self.invsqrtC, dx))**0.5)**2
            self.C = addouter(self.C, np.subtract(X[k], xold), wk * self.cmu / self.sigma**2)
            
        if self.counteval > self.update_eval + self.lazy_gap_evals:
            self.C = assert_symmetric(self.C)
            self.D, self.B = np.linalg.eig(self.C)
            for i in range(len(self.C)):
                for j in range(i+1):
                    self.invsqrtC[i][j] = self.invsqrtC[j][i] = sum(self.B[i][k] * self.B[j][k] / self.D[k]**0.5 for k in range(len(self.C)))
            self.update_eval = self.counteval
            
        cn, sum_square_ps = self.cs / self.damps, sum(x**2 for x in self.ps)
        self.sigma *= math.exp(min(1, cn * (sum_square_ps / self.dim - 1) / 2))
        
    def run(self, X, y):
        self.initialize_variables(X, y)
        
        while not self.stop():
            X = self.propose_candidates()
            X = self.sort_candidates(X)
            self.update_variables(X)
            
        return self.best_fitness
    
    def get_best_params(self):
        max_score = self.best_fitness[0]
        max_params = self.best[0]
        for i in range(len(self.best)):
            if self.best_fitness[i] >= max_score:
                max_score = self.best_fitness[i]
                max_params = self.best[i]
        return max_score, max_params

In [ ]:
class CMAESSearch:
    
    def __init__(self, estimators, estimator_params):
        self.estimators = estimators
        self.estimator_params = estimator_params
        self.results = {}
        
    def search_single(self, dataset):
        """ Perform a grid search using all estimators on a single dataset
        
        Args:
            dataset (tuple): a 4-tuple of X_train, X_test, y_train, y_test
        Returns:
            results dictionary with the best score, training time, and best 
            parameters for each estimator
        """
        
        X_train, X_test, y_train, y_test = dataset
        results = {}
        
        for estimator_name, estimator in self.estimators.items():
            print(estimator_name)
            params = self.estimator_params[estimator_name]
            cmaes = CMA_ES(params['default'], params['scale'], estimator)
            
            start = time.time()
            cmaes.run(X_train, y_train)
            end = time.time()
            
            best_score, best_params = cmaes.get_best_params()
            
            best_args = get_args(best_params, params['scale'], estimator)
            best_estimator = estimator(**best_args)
            best_estimator.fit(X_train, y_train)
            test_score = best_estimator.score(X_test, y_test)
            
            result_data = {'best_score': best_score,
                           'best_params': best_args,
                           'total_time': end-start,
                           'test_score': test_score}
            
            results[estimator] = result_data
            
        return results
    
    def search(self, datasets):
        for dataset_name, dataset in datasets.items():
            print(f'Searching {dataset_name}')
            dataset_results = self.search_single(dataset)
            self.results[dataset_name] = dataset_results
            print()
            
        return self.results
    
    def print_results(self):
        complete_search_time = 0
        
        # Find the estimator that performed best for each dataset
        for dataset_name, dataset_results in self.results.items():
            total_time = [estimator_results['total_time'] for estimator_results in dataset_results.values()]
            time_dict = {estimator_name: estimator_results['total_time'] for estimator_name, estimator_results in dataset_results.items()}
            dataset_time = sum(total_time)
            complete_search_time += dataset_time
            
            # Get a mapping from the best score to the estimator
            scores = [estimator_results['best_score'] for estimator_results in dataset_results.values()]
            score_to_estimator = {dataset_results[estimator]['best_score']: estimator for estimator in dataset_results.keys()}
            
            best_score = max(scores)
            best_estimator = score_to_estimator[best_score]            
            best_params = dataset_results[best_estimator]['best_params']
            best_test_score = dataset_results[best_estimator]['test_score']
            
            print(f'Dataset {dataset_name}: \nBest Estimator: {best_estimator} \n Params: {best_params} \nScore: {best_score} \nTotal time:{dataset_time} \nTesting score:{best_test_score} \nTime Dict:{time_dict}\n\n')
        
        results_df = pd.DataFrame(self.results)
        print('Grid Search total time:', complete_search_time)
        return results_df

In [ ]:
classification_estimators = {'Logistic Regression': LogisticRegression,
                             'KNN': KNeighborsClassifier,
                             'Decision Tree': DecisionTreeClassifier,
                             'AdaBoost': AdaBoostClassifier}
classification_parameters = {'Logistic Regression': {'default': [1, 1, 7.5],
                                                     'scale': {'C': 0, 'max_iter': 2, 'fit_intercept': -1}},
                             'KNN': {'default': [5, 3],
                                     'scale': {'n_neighbors': 0, 'leaf_size': 1}},
                             'Decision Tree': {'default': [2, 15],
                                               'scale': {'min_samples_split': 0, 'max_depth': 1}},
                             'AdaBoost': {'default': [5, 10],
                                          'scale': {'n_estimators': 1, 'learning_rate': -1}}}

regression_estimators = {'Linear Regression': LinearRegression,
                         'KNN': KNeighborsRegressor,
                         'Decision Tree': DecisionTreeRegressor,
                         'AdaBoost': AdaBoostRegressor,}
regression_parameters = {'Linear Regression': {'default': [7.5],
                                               'scale': {'fit_intercept': -1}},
                         'KNN': {'default': [5, 3],
                                     'scale': {'n_neighbors': 0, 'leaf_size': 1}},
                         'Decision Tree': {'default': [2, 15],
                                           'scale': {'min_samples_split': 0, 'max_depth': 1}},
                         'AdaBoost': {'default': [5, 10],
                                      'scale': {'n_estimators':6'learning_rate': -1}}}

In [ ]:
binary_data = load_data.load_binary()
binary_grid_search = CMAESSearch(classification_estimators, classification_parameters)

binary_grid_search.search(binary_data) 
binary_results = binary_grid_search.print_results()

In [ ]:
regression_data = load_data.load_regression()
regression_grid_search = CMAESSearch(regression_estimators, regression_parameters)

regression_grid_search.search(regression_data) 
regression_grid_search.print_results()

Dataset california_housing: 
Best Estimator: <class 'sklearn.tree._classes.DecisionTreeRegressor'> 
 Params: {'min_samples_split': 20, 'max_depth': 164.75119177399168} 
Score: 0.6669617209284268 
Total time:286.11477613449097 
Testing score:0.6891259025377072 
Time Dict:{<class 'sklearn.linear_model._base.LinearRegression'>: 0.13132405281066895, <class 'sklearn.neighbors._regression.KNeighborsRegressor'>: 8.299878358840942, <class 'sklearn.tree._classes.DecisionTreeRegressor'>: 34.90364217758179, <class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>: 242.77993154525757}


Dataset melbourne_housing: 
Best Estimator: <class 'sklearn.tree._classes.DecisionTreeRegressor'> 
 Params: {'min_samples_split': 0.014756092538863896, 'max_depth': 160.27228142977458} 
Score: 0.6565220457265545 
Total time:172.76335334777832 
Testing score:0.6973340219647486 
Time Dict:{<class 'sklearn.linear_model._base.LinearRegression'>: 0.16733789443969727, <class 'sklearn.neighbors._regression.KNeighbors

,california_housing,melbourne_housing,world_happiness
<class 'sklearn.linear_model._base.LinearRegression'>,"{'best_score': 0.6110921251096771, 'best_param...","{'best_score': -0.044998716689544395, 'best_pa...","{'best_score': 0.7404068030591904, 'best_param..."
<class 'sklearn.neighbors._regression.KNeighborsRegressor'>,"{'best_score': 0.1252408027018478, 'best_param...","{'best_score': 0.5188455495085263, 'best_param...","{'best_score': 0.6261647021338919, 'best_param..."
<class 'sklearn.tree._classes.DecisionTreeRegressor'>,"{'best_score': 0.6669617209284268, 'best_param...","{'best_score': 0.6565220457265545, 'best_param...","{'best_score': 0.6750782194363795, 'best_param..."
<class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>,"{'best_score': 0.5522534393640788, 'best_param...","{'best_score': 0.48764285967254306, 'best_para...","{'best_score': 0.7903462655546128, 'best_param..."


In [ ]:
multiclass_data = load_data.load_multiclass()
multiclass_grid_search = CMAESSearch(classification_estimators, classification_parameters)

multiclass_grid_search.search(multiclass_data) 
multiclass_grid_search.print_results()

Dataset mnist: 
Best Estimator: <class 'sklearn.neighbors._classification.KNeighborsClassifier'> 
 Params: {'n_neighbors': 5, 'leaf_size': 24.8611268924791} 
Score: 0.9805149617258176 
Total time:142.37768936157227 
Testing score:0.9861111111111112 
Time Dict:{<class 'sklearn.linear_model._logistic.LogisticRegression'>: 45.634217500686646, <class 'sklearn.neighbors._classification.KNeighborsClassifier'>: 1.107900857925415, <class 'sklearn.tree._classes.DecisionTreeClassifier'>: 2.887955904006958, <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>: 92.74761509895325}


Dataset forest_covertypes: 
Best Estimator: <class 'sklearn.neighbors._classification.KNeighborsClassifier'> 
 Params: {'n_neighbors': 4, 'leaf_size': 27.523529337036408} 
Score: 0.7833333333333333 
Total time:2168.0332849025726 
Testing score:0.814 
Time Dict:{<class 'sklearn.linear_model._logistic.LogisticRegression'>: 1836.6486015319824, <class 'sklearn.neighbors._classification.KNeighborsClassifier'>: 68.6

,mnist,forest_covertypes,kepler_exoplanets
<class 'sklearn.linear_model._logistic.LogisticRegression'>,"{'best_score': 0.9554627696590119, 'best_param...","{'best_score': 0.6371666666666665, 'best_param...","{'best_score': 0.7529719730568302, 'best_param..."
<class 'sklearn.neighbors._classification.KNeighborsClassifier'>,"{'best_score': 0.9805149617258176, 'best_param...","{'best_score': 0.7833333333333333, 'best_param...","{'best_score': 0.7551959375152125, 'best_param..."
<class 'sklearn.tree._classes.DecisionTreeClassifier'>,"{'best_score': 0.8343771746694503, 'best_param...","{'best_score': 0.7625000000000001, 'best_param...","{'best_score': 0.9792184021132301, 'best_param..."
<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>,"{'best_score': 0.6931106471816285, 'best_param...","{'best_score': 0.43566666666666665, 'best_para...","{'best_score': 0.98562296472228, 'best_params'..."
